# 

In [18]:
# unpack data
# !tar -xzvf JEOPARDY_QUESTIONS1.json.tar.gz

In [6]:
reset -fs

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import pandas as pd
import numpy as np
import networkx as nx

In [9]:
df = pd.read_json("JEOPARDY_QUESTIONS1.json")

In [10]:
df.head()

,category,air_date,question,value,answer,round,show_number
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",$200,Copernicus,Jeopardy!,4680
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,$200,Jim Thorpe,Jeopardy!,4680
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,$200,Arizona,Jeopardy!,4680
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",$200,McDonald\'s,Jeopardy!,4680
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",$200,John Adams,Jeopardy!,4680


In [11]:
df['air_date']= pd.to_datetime(df['air_date'])

In [12]:
# first round values, earliest show in data
# could also do show_number==1
df['air_date'].min(), df[(df['air_date']==df['air_date'].min()) & (df['round']=='Jeopardy!')]['value'].value_counts()

(Timestamp('1984-09-10 00:00:00'),
 $400    5
 $200    5
 $100    5
 $300    4
 $500    3
 $800    1
 Name: value, dtype: int64)

In [13]:
# example final jeopardy row, last show in data
df[(df['show_number']==4999) & (df['round']=='Final Jeopardy!')]

,category,air_date,question,value,answer,round,show_number
216929,HISTORIC NAMES,2006-05-11,'A silent movie title includes the last name o...,None,Grigori Alexandrovich Potemkin,Final Jeopardy!,4999


In [ ]:
# Existing category counts, desc
df.groupby('category').count()[['round']].sort_values(by=['round'], ascending=False).head(10)

,round
category,
BEFORE & AFTER,547
SCIENCE,519
LITERATURE,496
AMERICAN HISTORY,418
POTPOURRI,401
WORLD HISTORY,377
WORD ORIGINS,371
COLLEGES & UNIVERSITIES,351
HISTORY,349


Some cleanup/info
- [x] air-date to datetime
- [x] encode round
    - Daily Doubles are implicit from money value - Ignore
    - 3 Tiebreakers - Drop
- [ ] value to int/standardize money,  historical info [link](https://outsider.com/news/entertainment/jeopardy-show-double-dollar-values-categories/)
    - final question's have value np.nan
    - Rank question difficulty 1 through 11 (5 qn values 1st round, 5 qn values 2nd round, 1 final jeopardy)?
    - Or 1 through 13: 5 normal qn types plus daily double per round, times 2, and 1 final jeopardy
    - ~~for the pilot, questions values were 25-125 for first round, then doubled~~ pilot not in the data
    - Jeopardy! round was 100-500 for years
    - 11/26/01 doubled all monetary values, so 200-1000 for Jeopardy! round

In [14]:
df = df[df['round']!='Tiebreaker']

In [15]:
round_dict = {}
for idx, jp_round in enumerate(df['round'].unique()):
    round_dict[jp_round] = idx
round_dict

{'Jeopardy!': 0, 'Double Jeopardy!': 1, 'Final Jeopardy!': 2}

In [16]:
# df['enc_round'] = df['round'].map(round_dict)
df.replace({'round': round_dict}, inplace=True)

In [17]:
df.head(2)

,category,air_date,question,value,answer,round,show_number
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",$200,Copernicus,0,4680
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,$200,Jim Thorpe,0,4680


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=eebde281-32de-46a2-88ad-b5936664574a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>